In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
import cv2
import numpy as np
from os import listdir as ld
from os.path import join as pj
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.rpn import AnchorGenerator
from tqdm import tqdm

# Dataset
from dataset.detection.dataset import insects_dataset_from_voc_style_txt, collate_fn
# Loader
from IO.loader import load_path, load_images, load_annotations_path, load_annotations, get_anno_recs
# model
from model.faster_rcnn.faster_rcnn import make_Faster_RCNN

In [34]:
class args:
    # experiment name
    experiment_name = "crop_b2_2_4_8_16_32"
    # paths
    data_root = "/home/tanida/workspace/Insect_Phenology_Detector/data"
    test_image_root = "/home/tanida/workspace/Insect_Phenology_Detector/data/test_refined_images"
    model_root = pj("/home/tanida/workspace/Insect_Phenology_Detector/output_model/detection/Faster_RCNN", experiment_name)
    test_anno_folders = ["annotations_4"]
    # training config
    b_bone = "vgg16"
    anchor_size = ((2, 4, 8, 16, 32),)
    aspect_ratio = ((0.5, 1.0, 2.0),)
    input_size = 512 # choices=[320, 512, 1024]
    crop_num = (5, 5)
    max_insect_per_image = 20
    pretrain = True
    # class label
    labels = ['__background__','insects']

In [18]:
print('Loading dataset for test ...')
test_dataset = insects_dataset_from_voc_style_txt(args.test_image_root, args.input_size, args.crop_num, "Faster_RCNN", training=False)
test_data_loader = data.DataLoader(test_dataset, 1, num_workers=1, shuffle=False, collate_fn=collate_fn)
print('Loading annotation for test...')
annos, imgs = load_path(args.data_root, "refined_images", args.test_anno_folders)
images = load_images(imgs)
annotations_path = load_annotations_path(annos, images)
anno = load_annotations(annotations_path)
imagenames, recs = get_anno_recs(anno)

Loading dataset for test ...
Loading annotation for test...


In [35]:
print("Loading model for test ...")
model = make_Faster_RCNN(len(args.labels), args.input_size, args.anchor_size, args.aspect_ratio, args.b_bone, max_insect_per_image=args.max_insect_per_image, pretrain=args.pretrain).cuda()
load_name = pj(args.model_root, 'Faster_RCNN{}_{}.pth'.format(args.input_size, "final"))
model.load_state_dict(torch.load(load_name))

Loading model for test ...


<All keys matched successfully>

In [36]:
model

FasterRCNN(
  (transform): GeneralizedRCNNTransform()
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool